In [1]:
import numpy as np
from scipy.stats import norm

# --------------------------------------------------------------------------
# 1. DEFINIÇÃO DOS PARÂMETROS DO MERCADO E DA OPÇÃO
# --------------------------------------------------------------------------
S0 = 40      # Preço inicial do ativo (R$ 40)
K = 40       # Preço de exercício (Strike, R$ 40)
T = 0.5      # Tempo até a maturidade (6 meses = 0.5 anos)
rf = 0.05    # Taxa de juros livre de risco (5% a.a.)
sigma = 0.25 # Volatilidade do ativo (25% a.a.)
mu = 0.10    # Drift real do ativo (10% a.a.) - Usado apenas para fins de contexto.
             # Para precificação (risco-neutro), usamos rf.

# Parâmetros para a simulação de Monte Carlo
N = 1000000  # Número de simulações (um número alto para maior precisão)


# --------------------------------------------------------------------------
# 2. CÁLCULO PELO MODELO DE BLACK-SCHOLES
# --------------------------------------------------------------------------
# A fórmula de Black-Scholes é um padrão da indústria para precificar opções.

def black_scholes(S, K, T, r, sigma, option_type='call'):
    """
    Calcula o preço de uma opção europeia usando a fórmula de Black-Scholes.
    
    Parâmetros:
    S: Preço atual do ativo
    K: Preço de exercício (strike)
    T: Tempo até a maturidade (em anos)
    r: Taxa de juros livre de risco
    sigma: Volatilidade do ativo
    option_type: 'call' para opção de compra, 'put' para opção de venda
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == 'put':
        price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))
    else:
        raise ValueError("Tipo de opção inválido. Use 'call' ou 'put'.")
        
    return price

# Calculando os preços de Call e Put usando Black-Scholes
bs_call_price = black_scholes(S0, K, T, rf, sigma, option_type='call')
bs_put_price = black_scholes(S0, K, T, rf, sigma, option_type='put')


# --------------------------------------------------------------------------
# 3. CÁLCULO PELA SIMULAÇÃO DE MONTE CARLO
# --------------------------------------------------------------------------
# A simulação de Monte Carlo estima o preço da opção simulando milhares de
# possíveis preços futuros do ativo e calculando o payoff médio.

def monte_carlo_option_price(S, K, T, r, sigma, N, option_type='call'):
    """
    Calcula o preço de uma opção europeia usando a Simulação de Monte Carlo.
    
    Parâmetros:
    S: Preço atual do ativo
    K: Preço de exercício (strike)
    T: Tempo até a maturidade (em anos)
    r: Taxa de juros livre de risco (usada como drift na precificação risco-neutro)
    sigma: Volatilidade do ativo
    N: Número de simulações
    option_type: 'call' para opção de compra, 'put' para opção de venda
    """
    # Gera N números aleatórios de uma distribuição normal padrão
    Z = np.random.standard_normal(N)
    
    # Simula os preços finais do ativo (ST) usando o Movimento Browniano Geométrico
    # ST = S0 * exp((r - 0.5 * sigma^2) * T + sigma * sqrt(T) * Z)
    ST = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    
    # Calcula o payoff para cada preço simulado
    if option_type == 'call':
        payoff = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - ST, 0)
    else:
        raise ValueError("Tipo de opção inválido. Use 'call' ou 'put'.")
    
    # Calcula o preço da opção como a média dos payoffs trazida a valor presente
    price = np.exp(-r * T) * np.mean(payoff)
    
    return price

# Calculando os preços de Call e Put usando Monte Carlo
# Para garantir que os resultados sejam reproduzíveis, definimos uma semente aleatória.
np.random.seed(42) 
mc_call_price = monte_carlo_option_price(S0, K, T, rf, sigma, N, option_type='call')
mc_put_price = monte_carlo_option_price(S0, K, T, rf, sigma, N, option_type='put')


# --------------------------------------------------------------------------
# 4. EXIBIÇÃO DOS RESULTADOS
# --------------------------------------------------------------------------
print("--- Precificação de Opções Europeias ---")
print(f"Parâmetros: S0={S0}, K={K}, T={T} anos, rf={rf:.2%}, sigma={sigma:.2%}\n")

print("--- 1) Opção de Compra Europeia (Call) ---")
print(f"Preço pela fórmula de Black-Scholes: R$ {bs_call_price:.4f}")
print(f"Preço pela simulação de Monte Carlo ({N:,} simulações): R$ {mc_call_price:.4f}\n")

print("--- 2) Opção de Venda Europeia (Put) ---")
print(f"Preço pela fórmula de Black-Scholes: R$ {bs_put_price:.4f}")
print(f"Preço pela simulação de Monte Carlo ({N:,} simulações): R$ {mc_put_price:.4f}")

--- Precificação de Opções Europeias ---
Parâmetros: S0=40, K=40, T=0.5 anos, rf=5.00%, sigma=25.00%

--- 1) Opção de Compra Europeia (Call) ---
Preço pela fórmula de Black-Scholes: R$ 3.3040
Preço pela simulação de Monte Carlo (1,000,000 simulações): R$ 3.2987

--- 2) Opção de Venda Europeia (Put) ---
Preço pela fórmula de Black-Scholes: R$ 2.3164
Preço pela simulação de Monte Carlo (1,000,000 simulações): R$ 2.3194
